In [ ]:
############################################################################################################################
## Script: AGOLSQLSync.py
## Author: Jeremy Mullins, Derek Morgan
## Date: 06/05/2019
##
## Description:
##        This script is the test script for syncing data between ArcGIS Online
##        and an MS SQL Server database.
##
## Required prerequisites:
##        - ArcGIS API for Python
##            (https://developers.arcgis.com/python/guide/install-and-set-up/)
##
##        - pyodbc Module
##            (https://github.com/mkleehammer/pyodbc)
##                to install after download, open Python command
##                prompt and type the following:
##                    -- conda install pyodbc  --
##
##        - pandas Module
##            (https://pandas.pydata.org/pandas-docs/stable/)
##                to install after download, open Python command
##                prompt and type the following:
##                    -- conda install pandas --
##
##        - MS ODBC Driver for SQL Server (v17)
##            (https://docs.microsoft.com/en-us/sql/connect/odbc/download-odbc-driver-for-sql-server?view=sql-server-2017)
##
############################################################################################################################

In [ ]:
###########################################################################
## SECTION 1: ACCESSING FEATURE LAYER DATA, PREPARE CSV FOR SQL DATABASE ##
###########################################################################

# import all necessary modules
from arcgis.gis import GIS
from arcgis import geometry
from copy import deepcopy
import pyodbc as sql
import pandas as pd
import configparser
import zipfile
import os

# read config file
config = configparser.ConfigParser()
config.read('config_jm.ini')

# assign config variables
agolURL = config['AGOL']['URL']
agolUSER = config['AGOL']['USER']
agolPW = config['AGOL']['PW']
sqlDRVR = config['SQL']['SQLDRVR']
sqlSERV = config['SQL']['SERVER']
sqlDB = config['SQL']['DB']
sqlUSER = config['SQL']['USER']
sqlPW = config['SQL']['PW']
iD = config['SCRIPT']['itemID']
csvNAME = config['SCRIPT']['csvTITLE']
csvLOC = config['SCRIPT']['csvLOC']
zipLOC = config['SCRIPT']['zipLOC']
csvDOC = config['SCRIPT']['csvDOC']
newTEMP1 = config['SCRIPT']['newTEMP1']
newTEMP2 = config['SCRIPT']['newTEMP2']
csv2TEMP = config['SCRIPT']['csv2TEMP']
newINS1 = config['SCRIPT']['newINS1']
newINS2 = config['SCRIPT']['newINS2']
SQL2CSV = config['SCRIPT']['SQL2CSV']
sql2CSVout = config['SCRIPT']['sql2CSVout']
delTEMP1 = config['SCRIPT']['delTEMP1']
delTEMP2 = config['SCRIPT']['delTEMP2']

In [ ]:
# sign into AGOL acct
gis = GIS(agolURL,agolUSER,agolPW)

# get feature layer in question
featureLayer = gis.content.get(iD)

In [ ]:
# export feature layer as CSV (ZIP file)
output_file = featureLayer.export(title=csvNAME,export_format="CSV")
output_file.download(csvLOC)

In [ ]:
# unzip downloaded ZIP folder containing feature layer as CSV
zip_ref = zipfile.ZipFile(zipLOC,'r')
zip_ref.extractall(csvLOC)
zip_ref.close()

# delete ZIP folder on disk and CSV collection file in AGOL
os.remove(zipLOC)
output_file.delete()

In [ ]:
# read CSV using panda; replace NaNs with '000'
df = pd.read_csv(csvDOC)
df.fillna('000',inplace=True)

In [ ]:
######################################################
## SECTION 2: CONNECT TO SQL, PERFORM DATA ANALYSIS ##
######################################################

#Connect to SQL db and assign cursor
conn = sql.connect('Driver='+sqlDRVR+';'
                      'Server='+sqlSERV+';'
                      'Database='+sqlDB+';'
                      'trusted_connection=yes;'
                      'UID='+sqlUSER+';'
                      'PWD='+sqlPW+';')
cursor = conn.cursor()

In [ ]:
# create temporary tables in SQL db
cursor.execute(newTEMP1)
cursor.commit()

cursor.execute(newTEMP2)
cursor.commit()

In [ ]:
# insert CSV into temporary table
for index,row in df.iterrows():
    cursor.execute(csv2TEMP,row['FID'],
                   row['BUILDINGID'],
                   row['Descriptio'],
                   row['Classrooms'],
                   row['TypeCode'],
                   row['email'],
                   row['x'],
                   row['y']
                  )
cursor.commit()

In [ ]:
# compare tables and look for non-duplicated FIDs
cursor.execute(newINS1)
cursor.commit()

# compare tables and look for any updated records in prod table
cursor.execute(newINS2)
cursor.commit()

In [ ]:
##########################################################################
## SECTION 3: CONVERT SQL UPDATES TO CSV, PUSH TO AGOL, PERFORM UPDATES ##
##########################################################################

# export SQL data to CSV
outCSVscript = SQL2CSV

df2 = pd.read_sql_query(outCSVscript,conn)

df2.to_csv(sql2CSVout)

In [ ]:
# read the SQL update CSV
csv2 = sql2CSVout
updates_df_2 = pd.read_csv(csv2)

In [ ]:
# query all features in target AGOL feature layer
fl = featureLayer.layers[0]
flquery = fl.query()

In [ ]:
# determine which features overlap between update CSV and AGOL feature layer using an INNER join
overlap_rows = pd.merge(left = flquery.sdf, right = updates_df_2, how = 'inner', on = 'FID')

In [ ]:
# create list containing corrected features (ftbu)
updatefeatures = []
all_features = flquery.features

In [ ]:
# for loop to prepare updated geometries and attributes for each of the updated features
for fid in overlap_rows['FID']:
    # get the feature to be updated
    original_feature = [f for f in all_features if f.attributes['FID'] == fid][0]
    ftbu = deepcopy(original_feature)
    
    # get the matching row from csv
    matching_row = updates_df_2.where(updates_df_2.FID == fid).dropna()
    
    # get geometries in the destination coordinate system
    input_geometry = {'y':float(matching_row['Y']),
                      'x':float(matching_row['X'])}
    output_geometry = geometry.project(geometries = [input_geometry],
                                       in_sr = 4326,
                                       out_sr = flquery.spatial_reference['latestWkid'],
                                       gis = gis)
    # assign the updated values
    ftbu.geometry = output_geometry[0]
    ftbu.attributes['BUILDINGID'] = matching_row['BuildingID'].values[0]
    ftbu.attributes['Classrooms'] = matching_row['Classrooms'].values[0]
    ftbu.attributes['Descriptio'] = matching_row['Description'].values[0]
    ftbu.attributes['TypeCode'] = matching_row['TypeCode'].values[0]
    ftbu.attributes['FID'] = int(matching_row['FID'])
    ftbu.attributes['email'] = matching_row['email'].values[0]
    
    # add this to the list of features to be updated
    updatefeatures.append(ftbu)

In [ ]:
# call the edit_features() method of FeatureLayer object and pass features to the updates parameter
if len(updatefeatures) > 0:
    fl.edit_features(updates = updatefeatures)

In [ ]:
###########################################################
## SECTION 4: CLEAN UP DATA WITHIN SQL AND LOCAL MACHINE ##
###########################################################

# delete temporary table
cursor.execute(delTEMP1)
cursor.commit()

cursor.execute(delTEMP2)
cursor.commit()

In [ ]:
# close and delete cursor; close SQL db connection
cursor.close()
del cursor
conn.close()

In [ ]:
# remove all CSVs
os.remove(csvDOC)
os.remove(sql2CSVout)